# Speech Recognition with a SP network (control of strain)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import SP_anisotropy_class as SPN

import machine_learning_library.datasets.load_TI46 as TI46
from machine_learning_library.ridge_regression import *
from machine_learning_library.mask import *
from python_speech_features import mfcc

In [5]:
# Can specify the speakers to use or leave blank (or None) to include all
speakers = ['f1', 'f2', 'f3', 'f4', 'f5']

# Specifying digits_only=True and train=True returns only the spoken digits part of TI20 training set
# It returns the signal, label and sampling rate of the data
train_signal, train_label, train_rate, train_speaker = TI46.load_TI20(
        speakers, digits_only=True, train=True)

In [7]:
print(train_rate[0])
print(len(train_signal[0]))

12500
14848


In [ ]:
def pre_process(signals, rates, nfft=512):
    x = np.empty_like(signals)
    for i,(sig,rate) in enumerate(zip(signals, rates)):
        winlen = nfft / rate
        x[i] = mfcc(sig, rate, winfunc=np.hanning, winlen=winlen, winstep=winlen/2, nfft=nfft)
    return x

x_train = pre_process(train_signal, train_rate, nfft=512)
x_test = pre_process(test_signal, test_rate, nfft=512)

In [16]:
h = 0.4
theta_H = 90
k_s_0 = 0
phi = 45
f0 = 1e10
class Single_Node_Reservoir_Speech_Reco:
    
    def __init__(self, Nin, Nvirt, T_theta_ratio, m0, gamma, beta_prime = 10, bias = True):
        self.Nin = Nin
        self.Nvirt = Nvirt
        self.Nout = 10
        
        self.spn = SPN.SP_Network(h,theta_H,k_s_0,phi,beta_prime)
        SPN.calculate_energy_barriers(self.spn)
        self.T = 1./(self.spn.get_omega_prime()*f0)
        self.theta = self.T/T_theta_ratio
        self.tau = self.Nvirt*self.theta
        
        self.M = binary_mask(Nin,Nvirt,m0=m0)
        
        self.W = np.zeros((Nvirt,10))
        
        self.gamma = gamma
        
        self.use_bias = bias
    
    def gen_signal_fast_delayed_feedback(self, x, delay_fb):
        Ns = len(u)
        if self.use_bias:
            print("Use bias")
            S = np.zeros((Ns,self.Nvirt+1))
        else:
            S = np.zeros((Ns,self.Nvirt))
            
        for k in range(Ns):
            if k%100==0:
                print(k)
            for i in range(self.Nvirt):
                j = self.M[i]*u[k] + self.k_off #Offset
                self.spn.k_s = j + self.gamma*S[k-1,i-delay_fb] #Delayed Feedback 
                self.spn.evolve_fast(f0,self.theta)
                S[k,i] = self.spn.get_m_fast()
                
        if self.use_bias:
            for k in range(Ns):
                S[k,self.Nvirt] = 1
        
        return(S[spacer:])
    
    def gen_signal_delayed_feedback_without_SPN(self, u, delay_fb):
        Ns = len(u)
        if self.use_bias:
            print("Use bias")
            J = np.zeros((Ns,self.Nvirt+1))
        else:
            J = np.zeros((Ns,self.Nvirt))
            
        # Artificial nonlinearity
        #f = lambda x: x
        f=np.tanh
            
        for k in range(Ns):
            if k%100==0:
                print(k)
            for i in range(self.Nvirt):
                j = self.M[i]*u[k] + self.k_off #Offset
                J[k,i] = f(j + self.gamma*J[k-1,i-delay_fb]) #Delayed Feedback 
                
        if self.use_bias:
            for k in range(Ns):
                J[k,self.Nvirt] = 1
        
        return(J[spacer:])

NameError: name 'X' is not defined